# Introduction to Quantitative Finance

Copyright (c) 2019 Python Charmers Pty Ltd, Australia, <https://pythoncharmers.com>. All rights reserved.

<img src="img/python_charmers_logo.png" width="300" alt="Python Charmers Logo">

Published under the Creative Commons Attribution-NonCommercial 4.0 International (CC BY-NC 4.0) license. See `LICENSE.md` for details.

Sponsored by Tibra Global Services, <https://tibra.com>

<img src="img/tibra_logo.png" width="300" alt="Tibra Logo">


## Module 1.3: Ordinary Least Squares

### 1.3.2 Regression Tests

In this module we will look further into Multivariate OLS and examine some of the requirements of the algorithm, as well as some of the details of the regression results we saw in the last module.

When performing OLS for Linear Regression Models, there are a few assumptions that need to be met. The key ones are:

The first assumption is the key one - that is that the relationship between $X$ and $Y$ can, in fact, be described using the model $Y = X\beta + u$. It may *not* be able to be precisely modeled this way, but it may be possible to get close enough that it doesn't matter.

The second assumption is that the expected value of $u$ is zero. There may be fluctuations in the vector $u$, but the overall expected value is 0. More formally, we assume that $E(u|X) = 0$, that is the expected value of $u$ when given $X$ is zero. If it were not, then we can alter the bias term to make it zero, which would be learned from the OLS, giving us our zero value!

The third assumption is that the error term ($u$) and the data itself $X$ do not have any correlation. In other words, $u$ is unexplained error that cannot be explained by the data. Put more formally, there is no hetroskedasticity or autocorrelation between $u$ and $X$, which is a stronger assumption than the second, but along the same lines. We will cover these terms in a later module more formally.

The fourth assumption is that $X$ has a finite variance. This is sometimes (slightly incorrectly) referred to as $X$ being non-stochastic. We will investigate how variance plays into the model in several later modules.

The fifth assumption is that there are no linear relation between the measurements (variables, columns, features) in $X$, known as having **full column rank**.

If any of these assumptions are untrue, the resulting model does not necessarily have the properties we will discuss in the rest of this module, and the model itself might be biased or inaccurate. However, it may still be *useful* in a practical sense. For instance, if two variables are slightly linearly related, we break the last assumption, however in practice the model is generally still useful. However if they are heavily related, then the resulting model will be unstable.


<div class="alert alert-warning">
    Like most models and concepts, there is always some debate about the definitions and assumptions behind them. Further, some people use the same term to describe different concepts. When discussing an algorithm, it would be best practice to note any key assumptions or variance from the "norm" that you consider. If you aren't sure, provide a reference.
</div>


In [1]:
%run setup.ipy

Let's load in some data for a regression problem and have a look at the results. In this dataset, we are trying to predict house prices from other characteristics of the area, in Boston, Massachusetts. Prices are in thousands, but are from 1978, so are quite low!

In [2]:
# Let's load a dataset from the scikit learn repository
# scikit-learn is a machine learning library, and has a few sample datasets 
from sklearn.datasets import load_boston

In [3]:
boston_data = load_boston()

In [4]:
type(boston_data)

sklearn.utils.Bunch

In [5]:
print(boston_data.DESCR)

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

In [6]:
def sklearn_to_df(sklearn_dataset):
    # A helper function to convert the scikit-learn dataset to a pandas DataFrame
    # From: https://stackoverflow.com/questions/38105539/how-to-convert-a-scikit-learn-dataset-to-a-pandas-dataset/46379878#46379878
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

In [7]:
boston = sklearn_to_df(boston_data)

In [8]:
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [9]:
import statsmodels.formula.api as smf
est = smf.ols(formula='target ~ CRIM + ZN + INDUS + CHAS + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT', 
              data=boston).fit()  # Does the constant for us

In [10]:
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Fri, 26 Apr 2019   Prob (F-statistic):          6.95e-135
Time:                        15:06:13   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.4911      5.104      7.149      0.000      26.462      46.520
CRIM          -0.1072      0.033     -3.276      0.001      -0.171      -0.043
ZN             0.0464      0.014      3.380      0.001       0.019       0.073
INDUS          0.0209      0.061      0.339      0.735      -0.100       0.142
CHAS           2.6886      0.862      3.120      0.002       0.996       4.381
NOX          -17.7958      3.821     -4.658      0.000     -25.302     -10.289
RM             3.8048      0.418      9.102      0.000       2.983       4.626
AGE            0.0008      0.013      0.057      0.955      -0.025       0.027
DIS           -1.4758      0.199     -7.398      0.000      -1.868      -1.084
RAD            0.3057      0.066      4.608      0.000       0.175       0.436
TAX           -0.0123      0.004     -3.278      0.001      -0.020      -0.005
PTRATIO       -0.9535      0.131     -7.287      0.000      -1.211      -0.696
B              0.0094      0.003      3.500      0.001       0.004       0.015
LSTAT         -0.5255      0.051    -10.366      0.000      -0.625      -0.426
==============================================================================
Omnibus:                      178.029   Durbin-Watson:                   1.078
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              782.015
Skew:                           1.521   Prob(JB):                    1.54e-170
Kurtosis:                       8.276   Cond. No.                     1.51e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.51e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In the above table, there is a coef column, which gives the values for $\beta$ in our model for each independent variable.
If the coefficient is negative, there is an inverse relationship between the independent variable and the dependent one.
It is important to note that this is not a direct relationship, as retraining the model with just one parameter will likely change this coefficient:

In [11]:
import statsmodels.formula.api as smf
est_simple = smf.ols(formula='target ~ CRIM', 
              data=boston).fit()  # Does the constant for us

In [12]:
est_simple.params

Intercept    24.016220
CRIM         -0.412775
dtype: float64

In addition to the coefficient itself, we are given the standard error, the probability (using the t-statistic) that this value is significant (i.e. if it is less than 0.05), and the lower and upper bounds for the 95% confidence interval - where we can say with 95% confidence that the true value lies within those bounds.

A key reason for this is related to the second warning, indicating there is a strong multicollinearity. We will review this term in the next module and fix the problem it is causing over the next few modules. For now, it indicates that the independent variables are effectively correlated to a high degree, which breaks an assumption with OLS. In short, it means the independent variables are each predicting the same components of the output, and the coefficients are effectively arbitrary. 

As an example, if we have two variables $a$ and $b$ that are correlated, the coefficient value for $a$ and $b$ in a trained model is effectively shared between them, and whatever value actually appears in the OLS model is just one of many possibilities.

For the test statistics, good values (for various definitions of "good") for these scores allow us to say with a high confidence that the model accurately predicts the data. Bad values indicate that the model should not be used. We will now review a few key values from this table, as a means to validate our model.

### The $R^2$ statistic

The key statistic to review, and the "one value" that you are likely to report in your executive summary, is the $R^2$ statistic. It measures how much of the variance in the predicted variable ($Y$) is explained by your model ($X\beta$), compared to the error of the model ($u$). A high value (near 1) indicates that the model perfectly explains the variable being predicted. A low value (near 0) indicates that the model does not explain the variable at all, which is achieved if the model always predicts the expected value of $Y$. The score can be negative as well, as the model itself can be a net-negative in predictive power (i.e. it model actually predicts incorrectly more than correctly).

In the above results, the $R^2$ value is around 0.741, indicating that around 74% of the variance in the predicted variable $Y$ can be explained by the model $X\beta$. That said, our model has a few problems which we will address soon.

To obtain the $R^2$ value, store the regression results object obtained above and extract it:

In [13]:
est.rsquared

0.7406077428649428

#### Exercises

1. Review the documentation at the following link to see what other values can be obtained from a trained estimator:     http://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.RegressionResults.html#statsmodels.regression.linear_model.RegressionResults
2. What is the difference between `est.rsquared` and `est.rsquared_adj`? When should you use one over the other?

There are quite a few terms on the documentation page we haven't seen yet - many will be reviewed in later modules in this course.

### The $F$ statistic


The $F$ statistic is another measure of how significant the fit is. It divides the mean squared error of the model, by the mean squared error of the error term in the model. The probability value under it indicates the probability that we would achieve such a statistic, *if all the coefficients were zero*. In our model, our probability is very low (6.72e-135) indicating there is almost no chance that such an F statistic would be obtained by such a "zero" model.

In [14]:
est.fvalue

108.05702099860865

In [15]:
est.f_pvalue

6.946752831854574e-135

To put this formally, the F statistic is a test against the null hypothesis:

$H_0: \beta_i = 0 \forall i$

The alternative hypothesis is that *at least* one of the values in $\beta$ is not 0.

The F statistic can be computed using the following terms:

$ F = \frac{ESS}{RSS}$

Where $ESS$ is the explained variance of the model and $RSS$ is the unexplained variance. Given the explained variance of the model is due to the component $\beta X$ and the unexplained component is due to $u$, we can derive the equations as below:

$ ESS = \frac{1}{k-1}\sum{(\hat{Y_i} - \bar{Y})^2}$

Where $\hat{Y_i}$ is the *ith* predicted value and $\bar{Y}$ is the overall mean of $Y$, and $k$ is the number of variables. In other words, it is the total deviation from the mean that the model explains.

For the variance explained by the residuals, we get:

$ RSS = \frac{n}{k}\sum{u_i^2}$

Where $u$ is the error term in our linear regression model and $n$ is the number of samples. There are a few ways to alter these equations to make them easier to compute, all based on performing algebra with the OLS estimator equations defined in earlier modules.

### Likelihood Function, Akaike information criterion (AIC) and  Bayesian information criterion (BIC)

These three measures are related, and represent the plausibility of the given data given the set of parameters in the model.
In all three cases, we use them as relative values. That is, we use these values to compare two different models, and choose the model with the lowest score of these three values (or whichever single statistic you are most concerned with).

For instance, if model 1 has a BIC of 3085 and model 2 has a BIC of 4000, we choose model 1.

The key function here is the likelihood function, which is used to compute the AIC and BIC. The likelihood function $\mathcal{L}(\beta \mid x)$ is the likelihood that the data could be generated from a model with the given parameters. From an information theory perspective, we aim to maximise the likelihood function. From a computing perspective, it is often easier to both compute the *log likelihood*, and to *minimise the negative log likelihood*. A key component of this is that computers find adding numbers easier than multiplying small numbers, and we can convert from log-space to normal-space using the following pattern:

$log(xy) = log(x) + log(y)$

When dealing with probabilities, many probability values are very small, and multiplying small numbers near zero is hard for computers. Often, they "underflow" and consider a very small number to just be zero, and then any product from that point on is zero. Instead, we compute the log of all numbers and add them together - no underflow!

Once the likelihood function (or negative log likelihood) has been computed, the maximum value it can take (when optimised) is $\hat{L}$. From here, the AIC is defined as:

$ AIC = sk - s\ln(\hat{L})$

The BIC is defined similarly:

$ BIC = \ln(n)k - 2\ln({\hat L})$

Typically the BIC is preferred, as it is more stable in most circumstances. However, for the BIC to be valid, the number of samples must be much more than the number of parameters.